# Basic PDF extraction workflow
This is vastly oversimplified workflow that demonstrates the following:
1. Processing PDF into images by pages
2. Compute vector embeddings per page (using a pre-trained ViT)
3. We'll manually pick a page with a data table and get top10 similar pages to evaluate whether the embeddings are good

In [1]:
import locale
from pathlib import Path
import shutil

import ghostscript
import lance
import numpy as np
import pandas as pd
import pyarrow as pa
from PyPDF2 import PdfFileReader, PdfFileWriter

## Process the PDF
- PDF's can be both images and text
- For simplicity, we assume all pages are scanned images
- However we should also look at what is already text (saves OCR) when doing this for realz
- This only needs to be done once. Afterwards you can skip directly to the section on training using the saved Lance dataset

### Download

In [2]:
!curl https://www.hcd.ca.gov/housing-elements/docs/torrance-6th-adopted081522.pdf --output test.pdf
url = "https://www.hcd.ca.gov/housing-elements/docs/torrance-6th-adopted081522.pdf"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30.8M  100 30.8M    0     0  11.7M      0  0:00:02  0:00:02 --:--:-- 11.7M


### Split into pages

In [3]:
pages = Path(".") / "pages"
shutil.rmtree(pages)
pages.mkdir(exist_ok=True)


def write_page(i, page):
    writer = PdfFileWriter()
    writer.addPage(page)
    page_i = pages / f"page_{i}.pdf"
    with page_i.open("wb") as outfile:
        writer.write(outfile)
    return str(page_i.absolute())

page_paths = []
with open("test.pdf", 'rb') as fh:
    reader = PdfFileReader(fh)
    for i, page in enumerate(reader.pages):
        page_paths.append(write_page(i, page))

### Convert to image

In [4]:
!./install_deps.sh # ghostscript and tkinter

Running `brew update --auto-update`...
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).

You have 9 outdated formulae and 1 outdated cask installed.
You can upgrade them with brew upgrade
or list them with brew outdated.



In [5]:
images = Path(".") / "images"
shutil.rmtree(images)
images.mkdir(exist_ok=True)

def pdf2jpeg(pdf_input_path, jpeg_output_path):
    args = ["pdf2jpeg",
            "-dNOPAUSE",
            "-sDEVICE=jpeg",
            "-r144",
            f"-sOutputFile={jpeg_output_path}",
            pdf_input_path]
    encoding = locale.getpreferredencoding()
    args = [a.encode(encoding) for a in args]
    ghostscript.Ghostscript(*args)

for i, p in enumerate(page_paths):
    pdf2jpeg(p, str(images.absolute() / f"page_{i}.jpeg"))

GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.


/var/folders/p6/0wtfjg9d4bsd59fl0v2zgy3c0000gn/T/ipykernel_14850/2414296513.py:14: DeprecationWarning: Passing bytes-arguments to 'Ghostscript()' is deprecated
  ghostscript.Ghostscript(*args)


Processing pages 1 through 1.
Page 1
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages

openjpeg warning: unspec CS. 1 component so assuming gray.


GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1
GPL Ghostscript 

...

Loading NimbusSans-Regular font from %rom%Resource/Font/NimbusSans-Regular... 4377684 2894796 6472768 5123957 4 done.
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1


openjpeg warning: unspec CS. 1 component so assuming gray.


Loading NimbusSans-Regular font from %rom%Resource/Font/NimbusSans-Regular... 4377684 2894796 6472768 5123957 4 done.
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1
Loading NimbusSans-Regular font from %rom%Resource/Font/NimbusSans-Regular... 4377684 2889333 6853352 5513001 4 done.
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1
Loading NimbusSans-Regular font from %rom%Resource/Font/NimbusSans-Regular... 4377684 2894796 6472768 5123957 4 done.
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and com

openjpeg warning: unspec CS. 1 component so assuming gray.


Processing pages 1 through 1.
Page 1
Querying operating system for font files...
Substituting font Helvetica-Narrow-Bold for ArialNarrow-Bold.
Loading NimbusSansNarrow-Bold font from %rom%Resource/Font/NimbusSansNarrow-Bold... 4425876 2954795 6524648 5192647 4 done.
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1
Querying operating system for font files...
Substituting font Helvetica-Narrow-Bold for ArialNarrow-Bold.
Loading NimbusSansNarrow-Bold font from %rom%Resource/Font/NimbusSansNarrow-Bold... 4425876 2954795 6524648 5192647 4 done.
GPL Ghostscript 9.54.0 (2021-03-30)
Copyright (C) 2021 Artifex Software, Inc.  All rights reserved.
This software is supplied under the GNU AGPLv3 and comes with NO WARRANTY:
see the file COPYING for details.
Processing pages 1 through 1.
Page 1
Querying op

In [53]:
df = pd.DataFrame({
    "page_id": range(len(page_paths)),
    "combined": url,
    "page": page_paths,
    "image": pd.array(images.iterdir(), dtype='image[uri]')
})
df

,page_id,combined,page,image
0,0,https://www.hcd.ca.gov/housing-elements/docs/t...,/Users/changshe/code/pdf-extract/pages/page_0.pdf,images/page_1055.jpeg
1,1,https://www.hcd.ca.gov/housing-elements/docs/t...,/Users/changshe/code/pdf-extract/pages/page_1.pdf,images/page_606.jpeg
2,2,https://www.hcd.ca.gov/housing-elements/docs/t...,/Users/changshe/code/pdf-extract/pages/page_2.pdf,images/page_256.jpeg
3,3,https://www.hcd.ca.gov/housing-elements/docs/t...,/Users/changshe/code/pdf-extract/pages/page_3.pdf,images/page_743.jpeg
4,4,https://www.hcd.ca.gov/housing-elements/docs/t...,/Users/changshe/code/pdf-extract/pages/page_4.pdf,images/page_313.jpeg
...,...,...,...,...
1316,1316,https://www.hcd.ca.gov/housing-elements/docs/t...,/Users/changshe/code/pdf-extract/pages/page_13...,images/page_412.jpeg
1317,1317,https://www.hcd.ca.gov/housing-elements/docs/t...,/Users/changshe/code/pdf-extract/pages/page_13...,images/page_107.jpeg
1318,1318,https://www.hcd.ca.gov/housing-elements/docs/t...,/Users/changshe/code/pdf-extract/pages/page_13...,images/page_557.jpeg
1319,1319,https://www.hcd.ca.gov/housing-elements/docs/t...,/Users/changshe/code/pdf-extract/pages/page_13...,images/page_804.jpeg


In [54]:
shutil.rmtree("leoslittleshopofhorrors.lance")
lance.write_dataset(pa.Table.from_pandas(df), "leoslittleshopofhorrors.lance")

Next time we can just use the lance dataset in pandas / pytorch / duckdb directly from here out

## Plugging into pytorch

In [66]:
uri = "leoslittleshopofhorrors.lance"
pdf_dataset = lance.dataset(uri) # pyarrow Dataset
tbl = pdf_dataset.to_table()  # Arrow table
df = tbl.to_pandas()  # pandas dataframe
df.image[0]  # viewable in notebook

Image(images/page_1055.jpeg)

In [19]:
from transformers import ViTImageProcessor, ViTForImageClassification
from transformers.models.vit.modeling_vit import ViTEmbeddings

feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")

loading configuration file preprocessor_config.json from cache at /Users/changshe/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/1ba429d32753f33a0660b80ac6f43a3c80c18938/preprocessor_config.json
{param_name} should be a dictionary on of the following set of keys: {VALID_SIZE_DICT_KEYS}, got {size}. Converted to {size_dict}.
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

loading configuration file config.json from cache at /Users/changshe/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/5dca96d358b3fcb9d53b3d3881eb1ae20b6752d1/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures":

In [58]:
from lance.pytorch.data import Dataset
import torch

dataset = Dataset(uri, columns=["image"])
inputs = feature_extractor(df.image[0].to_pil(), return_tensors="pt")

# Add len(dataset)
embeddings = np.zeros((len(df), 197 * 768))
print(embeddings.shape)

with torch.no_grad():
    for i, batch in enumerate(dataset):
        inputs = feature_extractor(batch, return_tensors="pt")
        result = model.vit.embeddings(**inputs)
        embeddings[i, :] = result.numpy().ravel()

embeddings = pa.FixedSizeListArray.from_arrays(pa.array(embeddings.ravel()), list_size=768)

(1321, 151296)


In [69]:
arrays = [pa.array(np.repeat(df[c], 197)) for c in df.columns]
arrays.append(embeddings)

new_schema = pdf_dataset.schema.append(pa.field("embeddings", embeddings.type, False))
new_table = pa.Table.from_arrays(arrays, schema=new_schema)
lance.write_dataset(new_table, "to_patch_embeddings.lance")

# Notes

- Camelot and tabula-py only processes text pdf's
- PyPDF2 can't convert to image
- Installation is a pain
  - Camelot requires ghostscript and tkinter
  - Tabula-py requires JVM
